In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.
# os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

# import matplotlib.pyplot as plt

from dftorch import (
    Constants,
    Structure,
    StructureBatch,
    MDXL,
    MDXLBatch,
    MDXLOS,
    ESDriver,
    ESDriverBatch,
)

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""  # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)

torch.cuda.empty_cache()

# MD with PBC and PME
### Note, cell dimensions must be >= (2 x cutoff) if using PME.
### Also, do not use PME without PBC.

## First, SCF and forces

In [4]:
%%time
dftorch_params = {
    "coul_method": "FULL",  # 'FULL' for full coulomb matrix, 'PME' for PME method
    "Coulomb_acc": 5e-5,  # Coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 10.0,  # Coulomb cutoff
    "PME_order": 4,  # Ignored for FULL coulomb method
    "SCF_MAX_ITER": 100,  # Maximum number of SCF iterations
    "SCF_TOL": 1e-6,  # SCF convergence tolerance on density matrix
    "SCF_ALPHA": 0.2,  # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 20,  # Maximum Krylov subspace rank
    "KRYLOV_TOL": 1e-6,  # Krylov subspace convergence tolerance in SCF
    "KRYLOV_TOL_MD": 1e-4,  # Krylov subspace convergence tolerance in MD SCF
    "KRYLOV_START": 10,  # Number of initial SCF iterations before starting Krylov acceleration
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda" if torch.cuda.is_available() else "cpu"
# filename = '8RNT.xyz'
filename = "COORD.xyz"  # Solvated acetylacetone and glycine molecules in H20, Na, Cl
LBox = torch.tensor(
    [25.0, 25.0, 25.0], device=device
)  # Simulation box size in Angstroms. Only cubic boxes supported for now.
# Create constants container. Set path to SKF files.
const = Constants(
    filename,
    #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
    #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
    "/home/maxim/Projects/DFTB/DFTorch/experiments/sk_orig/mio-1-1/mio-1-1/",
    magnetic_hubbard_ldep=False,
).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=0, Te=1000.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)  # Calculate forces after SCF

torch.float64 torch.int64
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 1.4 s

### Do SCF ###
  Initial DM_Fermi
mu0 initial: tensor(0.1306, device='cuda:0')
Initial q_global: tensor([ 0.4275, -0.0774, -0.1562, -0.1117, -0.0861,  0.5134, -0.0738, -0.1386,
        -0.0923, -0.2120,  0.5105, -0.0720, -0.1408, -0.0931, -0.2101,  0.4532,
        -0.0873, -0.0649, -0.0665, -0.2219], device='cuda:0')

Starting cycle
Iter 1
Res = 0.174508520, dEc = 0.293518421, t = 0.1 s

Iter 2
Res = 0.133176856, dEc = 0.018412137, t = 0.0 s

Iter 3
Res = 0.101777728, dEc = 0.013239852, t = 0.0 s

Iter 4
Res = 0.077872751, dEc = 0.009640350, t = 0.0 s

Iter 5
Res = 0.059640177, dEc = 0.007087055, t = 0.0 s

Iter 6
Res = 0.045712857, dEc = 0.005248472, t = 0.0 s

Iter 7
Res = 0.035060743, dEc = 0.003909020, t = 0.0 s

Iter 8
Res = 0.026905114, dEc = 0.002924376, t = 0.0 s

Iter 9
Res = 0.020655517, dEc = 0.002195489, t = 0.0 s

Iter 10
Res = 0.015863134, dEc = 0.001652983, t = 0.0 s

In [3]:
# charges:
structure1.q

tensor([ 0.3670, -0.0733, -0.1296, -0.0958, -0.0732,  0.4510, -0.0760, -0.1167,
        -0.0820, -0.1783,  0.4479, -0.0762, -0.1190, -0.0816, -0.1757,  0.3879,
        -0.0824, -0.0521, -0.0561, -0.1859], device='cuda:0')

In [4]:
# MO coeffs
structure1.Q

tensor([[ 0.2980, -0.4211,  0.4328,  ..., -0.2183,  0.1995, -0.1116],
        [ 0.0685, -0.0474, -0.0530,  ..., -0.0033,  0.0567, -0.0569],
        [-0.0006,  0.0009, -0.0034,  ..., -0.0148,  0.0672, -0.0572],
        ...,
        [ 0.0609,  0.1394,  0.1946,  ...,  0.1111,  0.0836,  0.0372],
        [ 0.0674,  0.1524,  0.2084,  ...,  0.2065,  0.1584,  0.0778],
        [ 0.1522,  0.2504,  0.1777,  ...,  0.3384,  0.4118,  0.2963]],
       device='cuda:0')

In [5]:
print("Total Energy: ", structure1.e_tot)
print("Chemical Potential: ", structure1.mu0)

Total Energy:  tensor(-332.4864, device='cuda:0')
Chemical Potential:  tensor(0.3646, device='cuda:0')


## MD

In [6]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(
    structure1,
    dftorch_params,
    num_steps=5,
    dt=0.3,
    dump_interval=5,
    traj_filename="md_trj.xyz",
)

########## Step = 0 ##########
H0: 0.060 s
H1: 0.016 s
  rank: 0, Fel = 0.000048
KER: 0.008 s
F AND E: 0.003 s
ETOT = -332.48405318, EPOT = -332.59751719, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.1 s
0.02063872 GB


########## Step = 1 ##########
H0: 0.059 s
H1: 0.011 s
  rank: 0, Fel = 0.000074
KER: 0.007 s
F AND E: 0.002 s
ETOT = -332.48476160, EPOT = -332.91057338, EKIN = 0.11275559, T = 43.61575883, ResErr = 0.001167, t = 0.1 s
0.020639744 GB


########## Step = 2 ##########
H0: 0.060 s
H1: 0.014 s
  rank: 0, Fel = 0.000018
KER: 0.007 s
F AND E: 0.002 s
ETOT = -332.48664819, EPOT = -333.39432167, EKIN = 0.42392518, T = 163.98138812, ResErr = 0.002376, t = 0.1 s
0.020640768 GB


########## Step = 3 ##########
H0: 0.054 s
H1: 0.013 s
  rank: 0, Fel = 0.000011
KER: 0.006 s
F AND E: 0.002 s
ETOT = -332.48939016, EPOT = -334.00264643, EKIN = 0.90493151, T = 350.04272136, ResErr = 0.001327, t = 0.1 s
0.020641792 GB


########## Step = 4 ##########
H0: 0.054 s
H1: 0.014

# Batched MD with PBC and full Coulomb calculations.
### Systems in a batch should have same number of atoms and orbitals. Number of electrons and atom order can be arbitrary.
### Note, Batched mode does not support PME, only full Coulomb.

In [7]:
%%time
dftorch_params = {
    "coul_method": "FULL",
    "Coulomb_acc": 1e-5,  # coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 10.0,  # coulomb cutoff
    "PME_order": 4,
    "SCF_MAX_ITER": 20,
    "SCF_TOL": 1e-6,
    "SCF_ALPHA": 0.1,  # Linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 20,
    "KRYLOV_TOL": 1e-6,
    "KRYLOV_TOL_MD": 1e-4,
    "KRYLOV_START": 10,
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda"

filename = ["COORD_8WATER.xyz"] * 4
LBox = torch.tensor(
    [[21, 21, 21], [22, 22, 22], [21, 21, 22], [22, 22, 21]], device=device
)

const = Constants(
    filename, "/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/"
).to(device)

structure = StructureBatch(filename, LBox, const, charge=0, Te=1000.0, device=device)

es_driver = ESDriverBatch(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure, const, do_scf=True, verbose=False)
es_driver.calc_forces(structure, const)

int64
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-O.skf
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)

Starting cycle
Iter 1
Batch 0: Res = 1.474e+00, dEc = 2.097e-01
Batch 1: Res = 1.474e+00, dEc = 2.089e-01
Batch 2: Res = 1.474e+00, dEc = 2.092e-01
Batch 3: Res = 1.474e+00, dEc = 2.093e-01
Iter 2
Batch 0: Res = 1.242e+00, dEc = 8.997e-01
Batch 1: Res = 1.242e+00, dEc = 8.999e-01
Batch 2: Res = 1.242e+00, dEc = 8.998e-01
Batch 3: Res = 1.242e+00, dEc = 8.998e-01
Iter 3
Batch 0: Res = 1.047e+00, dEc = 7.246e-01
Batch 1: Res = 1.047e+00, dEc = 7.247e-01
Batch 2: Res = 1.047e+00, dEc = 7.246e-01
Batch 3: Res = 1.047e+00, dEc = 7.246e-01
Iter 4
Batch 0: Res = 8.831e-01, dEc = 5.870e-01
Batch 1: Res = 8.832e-01, dEc = 5.871e-01


In [8]:
torch.manual_seed(0)
temperature_K = torch.tensor([20.0, 30.0, 30.0, 30.0], device=structure.device)
mdDriver = MDXLBatch(es_driver, const, temperature_K=temperature_K)
mdDriver.run(
    structure,
    dftorch_params,
    num_steps=5,
    dt=0.3,
    dump_interval=5,
    traj_filename="batch_md_trj.xyz",
)

########## Step = 0 ##########
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)
H0: 1.000 s
H1: 0.009 s
  rank: 0, batch 0, Fel = 0.000056
  rank: 0, batch 1, Fel = 0.000059
  rank: 0, batch 2, Fel = 0.000041
  rank: 0, batch 3, Fel = 0.000044
  Not converged: 0
KER: 0.003 s
F AND E: 0.002 s
ETOT = -873.74561658, EPOT = -873.94177829, EKIN = 0.05687440, T = 18.33331009, ResErr = 0.000000, t = 1.0 s
ETOT = -873.71672304, EPOT = -873.93116957, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.0 s
ETOT = -873.71691834, EPOT = -873.89832729, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.0 s
ETOT = -873.71698197, EPOT = -873.90553529, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.0 s
0.023995904 GB


########## Step = 1 ##########
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)
H0: 0.987 s
H1: 0.009 s
  rank: 0, batch 0, Fel = 0.000080
  rank: 0, batch 1, Fel = 0.000081
  rank: 0, batch 2, Fel = 0.000079
  rank: 0, 

# 2. MD without PBC and with full Coulomb
### Do not use PME without PBC.
### Turn on PBC if necessary
## First, SCF and forces

In [9]:
%%time
dftorch_params = {
    "coul_method": "FULL",  # 'FULL' for full coulomb matrix, 'PME' for PME method
    "Coulomb_acc": 5e-5,  # Coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 12.0,  # Coulomb cutoff
    "PME_order": 4,  # Ignored for FULL coulomb method
    "SCF_MAX_ITER": 100,  # Maximum number of SCF iterations
    "SCF_TOL": 1e-6,  # SCF convergence tolerance on density matrix
    "SCF_ALPHA": 0.1,  # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 20,  # Maximum Krylov subspace rank
    "KRYLOV_TOL": 1e-6,  # Krylov subspace convergence tolerance in SCF
    "KRYLOV_TOL_MD": 1e-4,  # Krylov subspace convergence tolerance in MD SCF
    "KRYLOV_START": 10,  # Number of initial SCF iterations before starting Krylov acceleration
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda" if torch.cuda.is_available() else "cpu"
# filename = '8RNT.xyz'
filename = "COORD.xyz"  # Solvated acetylacetone and glycine molecules in H20, Na, Cl
# LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
LBox = None
# Create constants container. Set path to SKF files.
const = Constants(
    filename,
    #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
    #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\mio-1-1\\mio-1-1\\',
    "/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/",
    magnetic_hubbard_ldep=False,  # atom/shell-resolved spin parameters w for open-shell.
).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=0, Te=600.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)  # Calculate forces after SCF

int64
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-C.skf
torch.float64 torch.int64
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 1.909176454, dEc = 1.947740024, t = 0.0 s

Iter 2
Res = 1.378555914, dEc = 0.531737232, t = 0.0 s

Iter 3
Res = 0.994133558, dEc = 0.329875968, t = 0.0 s

Iter 4
Res = 0.716710558, dEc = 0.210435680, t = 0.0 s

Iter 5
Res = 0.516895688, dEc = 0.137885282, t = 0.0 s

Iter 6
Res = 0.373106348, dEc = 0.092571922, t = 0.0 s

Iter 7
Res = 0.269664262, dEc = 0.063480285, t = 0.0 s

Iter 8
Res = 0.195243480, dEc = 0.044317646, t = 0.0 s

Iter 9
Res = 0.141685354, dEc = 0.031404017, t = 0.0 s

Iter 10
Res = 0.103121096, dEc = 0.022529893, t = 0.0 s

Iter 11
  rank: 0, Fe

In [10]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(
    structure1,
    dftorch_params,
    num_steps=5,
    dt=0.3,
    dump_interval=5,
    traj_filename="md_trj.xyz",
)

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.063 s
H1: 0.001 s
  rank: 0, Fel = 0.000094
KER: 0.002 s
F AND E: 0.002 s
ETOT = -331.68029672, EPOT = -331.80866399, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.1 s
0.023874048 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.054 s
H1: 0.001 s
  rank: 0, Fel = 0.000239
  rank: 1, Fel = 0.000020
KER: 0.003 s
F AND E: 0.002 s
ETOT = -331.68116571, EPOT = -332.16345363, EKIN = 0.12749828, T = 49.31847879, ResErr = 0.001205, t = 0.1 s
0.02387456 GB


########## Step = 2 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.053 s
H1: 0.001 s
  rank: 0, Fel = 0.000110
  rank: 1, Fel = 0.000012
KER: 0.003 s
F AND E: 0.002 s
ETOT = -331.68339822, EPOT = -332.70815608, EKIN = 0.48005540, T = 185.69349980, ResErr = 0.002534, t = 0.1 s
0.023875072 GB


########## Step = 3 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.055 s
H1:

# 3. Open-Shell MD.
### Turn on PBC if necessary
### No batch mode for open-shell
## First, SCF and forces

In [11]:
%%time
dftorch_params = {
    "UNRESTRICTED": True,
    "BROKEN_SYM": True,  # if True, mix 2 % of lumo in homo at initialization
    "coul_method": "!PME",  # 'FULL' for full coulomb matrix, 'PME' for PME method
    "Coulomb_acc": 5e-5,  # Coulomb accuracy for full coulomb calcs or t_err for PME
    "cutoff": 12.0,  # Coulomb cutoff
    "PME_order": 4,  # Ignored for FULL coulomb method
    "SCF_MAX_ITER": 100,  # Maximum number of SCF iterations
    "SCF_TOL": 5e-6,  # SCF convergence tolerance on density matrix
    "SCF_ALPHA": 0.1,  # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.
    "KRYLOV_MAXRANK": 15,  # Maximum Krylov subspace rank
    "KRYLOV_TOL": 9e-6,  # Krylov subspace convergence tolerance in SCF
    "KRYLOV_TOL_MD": 1e-4,  # Krylov subspace convergence tolerance in MD SCF
    "KRYLOV_START": 10,  # Number of initial SCF iterations before starting Krylov acceleration
}

# Initial data, load atoms and coordinates, etc in COORD.dat
device = "cuda" if torch.cuda.is_available() else "cpu"
# filename = '8RNT.xyz'
filename = "O2.xyz"
LBox = None
# LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.

# Create constants container. Set path to SKF files.
const = Constants(
    filename,
    #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
    #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
    "/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/",
    magnetic_hubbard_ldep=False,
).to(device)

# Create structure object. Define total charge and electronic temperature.
# charge=0 and spin_pol need to be specified for open-shell systems.
# spin_pol is the number of unpaired electrons, not multiplicity.
# Shared chemical potential is used for both spin channels, so spin_pol has no effect but needs to be set anyways.
structure1 = Structure(
    filename,
    LBox,
    const,
    charge=0,
    spin_pol=0,
    os=dftorch_params["UNRESTRICTED"],
    Te=500.0,
    device=device,
)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(
    dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device
)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)  # Calculate forces after SCF

int64
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/O-O.skf
torch.float64 torch.int64
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 0.076646666, dEc = 0.001485381, t = 0.0 s

Iter 2
Res = 0.134471635, dEc = 0.000282222, t = 0.0 s

Iter 3
Res = 0.233454065, dEc = 0.000228600, t = 0.0 s

Iter 4
Res = 0.388443024, dEc = 0.000185166, t = 0.0 s

Iter 5
Res = 0.580101469, dEc = 0.000149985, t = 0.0 s

Iter 6
Res = 0.717176839, dEc = 0.000121487, t = 0.0 s

Iter 7
Res = 0.738018734, dEc = 0.000098405, t = 0.0 s

Iter 8
Res = 0.692595307, dEc = 0.000079708, t = 0.0 s

Iter 9
Res = 0.630998430, dEc = 0.000064563, t = 0.0 s

Iter 10
Res = 0.570071391, dEc = 0.000052296, t = 0.0 s

Iter 11
  rank: 0, Fel = 0.000000
Res = 0.513742896, dEc = 0.000222948, t = 0.0 s

Iter 12
  rank: 0, Fel = 0.000000
Res = 0.000394597, dEc = 0.000000000, t = 0.0 s

Iter 13
zero norm_dr
Res = 0.000000000, dEc = 0.000000000, t = 0.0 s

C

In [12]:
structure1.e_tot, structure1.e_spin

(tensor(-176.9905, device='cuda:0'), tensor(-0.7565, device='cuda:0'))

In [13]:
structure1.q_tot_atom

tensor([-1.0103e-14, -2.9976e-15], device='cuda:0')

In [14]:
# net spin per shell
structure1.net_spin_sr

tensor([ 0.0000, -1.0000,  0.0000, -1.0000], device='cuda:0')

In [15]:
# net spin (sums to a number of unpolarized electrons)
structure1.net_spin_sr.sum()

tensor(-2.0000, device='cuda:0')

## MD

In [16]:
torch.manual_seed(0)
temperature_K = torch.tensor(300.0, device=structure1.device)
mdDriver = MDXLOS(es_driver, const, temperature_K=temperature_K)
# mdDriver.NoRank = True
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(
    structure1,
    dftorch_params,
    num_steps=5,
    dt=0.2,
    dump_interval=5,
    traj_filename="md_trj.xyz",
)

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.070 s
H1: 0.003 s
  rank: 0, Fel = 0.000000
KER: 0.004 s
F AND E: 0.003 s
ETOT = -176.95167202, EPOT = -176.98782018, EKIN = 0.03877800, T = 149.99980985, ResErr = 0.000000, t = 0.1 s
0.021709312 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.051 s
H1: 0.002 s
  rank: 0, Fel = 0.000000
KER: 0.003 s
F AND E: 0.003 s
ETOT = -176.95166954, EPOT = -176.98497813, EKIN = 0.03615064, T = 139.83674359, ResErr = 0.001184, t = 0.1 s
0.021709824 GB


########## Step = 2 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.050 s
H1: 0.002 s
  rank: 0, Fel = 0.000000
KER: 0.003 s
F AND E: 0.004 s
ETOT = -176.95166674, EPOT = -176.98196970, EKIN = 0.03331139, T = 128.85403952, ResErr = 0.000171, t = 0.1 s
0.021710336 GB


########## Step = 3 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.052 s
H1: 0.002 s
  rank: 0, Fel = 0.000000
KER: 0.003 s
